In [252]:
import pandas as pd
import pdfplumber

In [253]:
def cb_gdp_report_parser(file_path):
    with pdfplumber.open(file_path) as pdf:
        first_page = pdf.pages[0]
        pdf_text   = first_page.extract_text()

    gdp_row_data = pdf_text.split('Валовой внутренний продукт')[1].split('\n')[0].replace(",", ".").split(' ')

    gdp_values = []
    for i in gdp_row_data:
        if 3 <= len(i) <= 5:
            gdp_values.append(float(i))
        elif len(i) == 7:
            min = float(i.split('-')[0])
            max = float(i.split('-')[1])
            gdp_values.append((min+max)/2)
        elif len(i) == 0:
            continue
        else:
            if i.startswith('('):
                min   = float(i.split('(')[1].split(')')[0])
                i_end = i.split(')-')[1]
            else:
                min   = float(i.split('-')[0])
                i_end = i[4:]          
            if i_end.startswith('('):
                max = float(i_end.split('(')[1].split(')')[0])
            else:
                max = float(i_end)
            gdp_values.append((min+max)/2)

    columns=[]
    for i in range(len(gdp_values)):
        columns.append(int('20'+file_path[-10:][:2])-1+i)

    gdp = pd.DataFrame(gdp_values). T
    gdp.columns = columns
    
    return gdp